In [98]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from dotenv import load_dotenv
import os, warnings
warnings.filterwarnings('ignore')
from langchain.output_parsers import CommaSeparatedListOutputParser, DatetimeOutputParser, OutputFixingParser, PydanticOutputParser
from pydantic import BaseModel, Field

In [4]:
load_dotenv()
api_key = os.environ['OPENAI_API_KEY']

In [8]:
model = ChatOpenAI(api_key=api_key)

In [12]:
output_parser = CommaSeparatedListOutputParser()
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [14]:
reply = "red, green, blue"

In [16]:
output_parser.parse(reply)

['red', 'green', 'blue']

In [22]:
human_template = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)
human_prompt

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'request'], input_types={}, partial_variables={}, template='{request}\n{format_instructions}'), additional_kwargs={})

In [24]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['format_instructions', 'request'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'request'], input_types={}, partial_variables={}, template='{request}\n{format_instructions}'), additional_kwargs={})])

In [30]:
model_request = chat_prompt.format_prompt(request="Give me 5 characteristics of dogs",format_instructions=output_parser.get_format_instructions()).to_messages()
model_request

[HumanMessage(content='Give me 5 characteristics of dogs\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`', additional_kwargs={}, response_metadata={})]

In [32]:
result = model(model_request)
print(result.content)

Loyal, playful, friendly, protective, intelligent


In [34]:
output_parser.parse(result.content)

['Loyal', 'playful', 'friendly', 'protective', 'intelligent']

In [36]:
model_request = chat_prompt.format_prompt(request="Write an article on dogs",format_instructions=output_parser.get_format_instructions()).to_messages()
model_request

[HumanMessage(content='Write an article on dogs\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`', additional_kwargs={}, response_metadata={})]

In [38]:
result = model(model_request)
print(result.content)

Loyal, intelligent, playful, affectionate, protective, social, energetic, trainable, obedient, faithful, loving, fun-loving, furry, cute, cuddly, furry, active, companionable, friendly, loyal, faithful, adventurous, loyal, sociable, protective, agile, curious, obedient, alert, clever, devoted, gentle, outgoing, sociable, spirited, stubborn, territorial, watchful, adorable, mischievous, independent, loyal, strong, vibrant, versatile, sweet, loving, reliable, devoted, brave, affectionate, smart, agile, protective, inquisitive, loving, obedient, happy, playful, loving, energetic, bright, devoted, trainable, loyal, friendly, brave, strong, curious, affectionate, protective, loyal, obedient.


In [40]:
output_parser.parse(result.content)

['Loyal',
 'intelligent',
 'playful',
 'affectionate',
 'protective',
 'social',
 'energetic',
 'trainable',
 'obedient',
 'faithful',
 'loving',
 'fun-loving',
 'furry',
 'cute',
 'cuddly',
 'furry',
 'active',
 'companionable',
 'friendly',
 'loyal',
 'faithful',
 'adventurous',
 'loyal',
 'sociable',
 'protective',
 'agile',
 'curious',
 'obedient',
 'alert',
 'clever',
 'devoted',
 'gentle',
 'outgoing',
 'sociable',
 'spirited',
 'stubborn',
 'territorial',
 'watchful',
 'adorable',
 'mischievous',
 'independent',
 'loyal',
 'strong',
 'vibrant',
 'versatile',
 'sweet',
 'loving',
 'reliable',
 'devoted',
 'brave',
 'affectionate',
 'smart',
 'agile',
 'protective',
 'inquisitive',
 'loving',
 'obedient',
 'happy',
 'playful',
 'loving',
 'energetic',
 'bright',
 'devoted',
 'trainable',
 'loyal',
 'friendly',
 'brave',
 'strong',
 'curious',
 'affectionate',
 'protective',
 'loyal',
 'obedient.']

In [42]:
model_request = chat_prompt.format_prompt(request="What is the best dog",format_instructions=output_parser.get_format_instructions()).to_messages()
model_request

[HumanMessage(content='What is the best dog\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`', additional_kwargs={}, response_metadata={})]

In [46]:
output_parser.parse(model(model_request).content)

['Labrador Retriever',
 'German Shepherd',
 'Golden Retriever',
 'French Bulldog',
 'Poodle',
 'Beagle',
 'Bulldog',
 'Australian Shepherd',
 'Siberian Husky',
 'Dachshund']

In [48]:
model_request = chat_prompt.format_prompt(request="Write a poem about animals",format_instructions=output_parser.get_format_instructions()).to_messages()
model_request

[HumanMessage(content='Write a poem about animals\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`', additional_kwargs={}, response_metadata={})]

In [50]:
output_parser.parse(model(model_request).content)

['Lions',
 'tigers',
 'bears',
 'oh my',
 'Elephants',
 'monkeys',
 'birds that fly',
 'Dolphins',
 'whales',
 'creatures of the sea',
 'Wolves',
 'foxes',
 'animals wild and free']

In [54]:
output_parser = DatetimeOutputParser()
output_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0771-05-13T01:44:26.894958Z, 1427-06-14T04:07:20.990028Z, 1868-07-06T00:54:57.032059Z\n\nReturn ONLY this string, no other words!"

In [66]:
# system_template = "Your reply to questions should always be in datetime format"

In [68]:
template_text = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)
# system_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [70]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [72]:
model_request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",format_instructions=output_parser.get_format_instructions()).to_messages()
model_request

[HumanMessage(content='Your reply to questions should always be in datetime format', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What date was the 13th Amendment ratified in the US?\nWrite a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1595-07-23T17:09:24.192856Z, 0687-07-03T20:21:48.718906Z, 1820-11-04T05:15:16.619232Z\n\nReturn ONLY this string, no other words!", additional_kwargs={}, response_metadata={})]

In [74]:
result = model(model_request,temperature=0)
print(result.content)

1865-12-06T00:00:00.000000Z


In [76]:
output_parser.parse(result.content)

datetime.datetime(1865, 12, 6, 0, 0)

In [86]:
misformatted = f'The 13th Amendment was ratified in the US on {result.content}. The datetime string matching the given pattern is: "1865-12-06T00:00:00.000000Z"'
misformatted

'The 13th Amendment was ratified in the US on 1865-12-06T00:00:00.000000Z. The datetime string matching the given pattern is: "1865-12-06T00:00:00.000000Z"'

In [88]:
output_parser

DatetimeOutputParser()

In [92]:
new_parser = OutputFixingParser.from_llm(parser=output_parser,llm=model)

In [94]:
new_parser.parse(misformatted)

datetime.datetime(1865, 12, 6, 0, 0)

In [100]:
class Scientist(BaseModel):
    name: str = Field(description='Name of a Scientist')
    discoveries: list = Field(description='List of scientific discoveries')

In [102]:
parser = PydanticOutputParser(pydantic_object=Scientist)
parser

PydanticOutputParser(pydantic_object=<class '__main__.Scientist'>)

In [104]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of a Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "List of scientific discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}
```


In [106]:
human_prompt = HumanMessagePromptTemplate.from_template("{request}\n{format_instructions}")

In [108]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [114]:
model_request = chat_prompt.format_prompt(request='Tell me about a famous scientist',format_instructions=parser.get_format_instructions()).to_messages()
model_request

[HumanMessage(content='Tell me about a famous scientist\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of a Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "List of scientific discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}\n```', additional_kwargs={}, response_metadata={})]

In [116]:
result = model(model_request,temperature=0.5)
print(result.content)

{
  "name": "Albert Einstein",
  "discoveries": ["Theory of Relativity", "Photoelectric Effect", "Mass-energy Equivalence"]
}


In [118]:
result

AIMessage(content='{\n  "name": "Albert Einstein",\n  "discoveries": ["Theory of Relativity", "Photoelectric Effect", "Mass-energy Equivalence"]\n}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 200, 'total_tokens': 233, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-86bf9012-98e5-40f8-96cb-d4abcec0c096-0')

In [122]:
type(parser.parse(result.content))

__main__.Scientist